In [40]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [41]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [42]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [43]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [44]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [45]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [46]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [47]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [48]:
vid = 'V570882'
token = '1df91d828519e519b420639f24c9b42e'

In [49]:
transaction = None
#Get 1 transaction
for message in txconsumer:
    transaction = message.value
    break
print(transaction)

{'txid': 'TX00123', 'payer': 'A99662', 'payee': 'A39529', 'amount': 1763}


In [50]:
#Generate signature based on given data
signature = gen_signature(transaction['txid'],transaction['payer'], transaction['payee'], transaction['amount'], token)
print(signature)

e4173e0a5abe083e1c4ee3f03f9016c8


In [51]:
#Prepare data to be serialize
to_submit = {'vid': vid, 'txid': transaction['txid'], 'signature': signature}
print(to_submit)

#Serialize data
data = serialize(submitschema, to_submit)
print(data)

#Submit data
producer.send('submit', value=data)

{'vid': 'V570882', 'txid': 'TX00123', 'signature': 'e4173e0a5abe083e1c4ee3f03f9016c8'}
b'\x0eV570882\x0eTX00123@e4173e0a5abe083e1c4ee3f03f9016c8'


In [52]:
result = None

#Get a result that matches with vid and txid
for message in resultconsumer:
    if message.value['vid'] == vid and message.value['txid'] == transaction['txid']:
        print(message.value)
        break

{'timestamp': 1731658674, 'vid': 'V570882', 'txid': 'TX00123', 'checksum': '58845abba0e2a6b6d1d826702520fa28', 'code': 200, 'message': 'Confirm'}
